<a href="https://colab.research.google.com/github/SofiazaR/Multilevel-information-model/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

In [1]:
import os

import requests
from bs4 import BeautifulSoup

sites = ['https://samplefocus.com/tag/memphis','https://memphis.dev/']

links = []
for i in sites:
    if not i in links:
        links.append(i)

count = 0
i = 0

if not os.path.exists(fr'texts'):
    os.makedirs(fr'texts')

while count < 100:
    if not links[i] in links:
        i += 1
        continue
    current_site = links[i]
    i += 1

    try:
        r = requests.get(current_site)
    except:
        print(f'Ошибка при запросе на сайт: {current_site}')
        continue

    soup = BeautifulSoup(r.text, 'lxml')
    text = soup.get_text()

    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)

    num_words = len(text.split())

    all_a = soup.find_all('a')

    for a in all_a:
        if not links.__contains__(a.get('href')):
            links.append(a.get('href'))
    if num_words >= 1000:
        count += 1
        with open(f"texts/{count}.txt", "w", encoding="utf-8") as myfile:
            myfile.write(text)
            myfile.close()
        with open(fr"index.txt", "a") as myfile:
            myfile.write(f"{count}.txt -> {current_site}\n")
    print(count)

print("Finished")

0
0
Ошибка при запросе на сайт: /
Ошибка при запросе на сайт: #
Ошибка при запросе на сайт: /categories
Ошибка при запросе на сайт: /tag
Ошибка при запросе на сайт: /collections
Ошибка при запросе на сайт: /samples/new
Ошибка при запросе на сайт: /about
0
Ошибка при запросе на сайт: /users/sign_in
Ошибка при запросе на сайт: /users/sign_up
0
0
0
0
0
0
Ошибка при запросе на сайт: /privacy
Ошибка при запросе на сайт: /cdn-cgi/l/email-protection#d1b9b4bdbdbe91a2b0bca1bdb4b7beb2a4a2ffb2bebc
0
0
Ошибка при запросе на сайт: #/portal/signin
Ошибка при запросе на сайт: #/portal/signup
Ошибка при запросе на сайт: /blog/essential-tips-ableton-live-11/
Ошибка при запросе на сайт: /blog/the-secret-to-perfect-transitions/
Ошибка при запросе на сайт: /blog/rave-hits-pack/
Ошибка при запросе на сайт: /blog/bootleg-remixing/
Ошибка при запросе на сайт: /blog/ableton-live-shortcut-list/
Ошибка при запросе на сайт: /blog/drum-machine-magic-create-beats-drum-patterns-that-stick/
Ошибка при запросе на сай

# Задание 2

In [2]:
pip install pymorphy3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 49.0 MB/s eta 0:00:00


In [3]:
import pymorphy3  # библиотека, которая может взять начальную форму слов
import glob
import io
import os
import string
import unicodedata
import re
import nltk
from nltk.stem import WordNetLemmatizer
from pymystem3 import Mystem



morph = pymorphy3.MorphAnalyzer()

# CONJ- союз
# INTJ- междометие
functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}  # function words - предлоги и незначащие слова
path_of_files = "texts/"


def pos(word, morth=pymorphy3.MorphAnalyzer()):
    "Return a likely part of speech for the *word*."""
    return morth.parse(word)[0].tag.POS


def letters(text):
    return ''.join(filter(str.isalpha, text))


def lemmatize(words, tokenized_map):
    for word in words:
        if pos(word) not in functors_pos:
            p = morph.parse(word)[0].normal_form
            arr = tokenized_map.get(p)
            wnl = WordNetLemmatizer()
            if arr is None:
                new_arr = set()
                new_arr.add(word)
                tokenized_map[p] = new_arr
            else:
                arr.add(word)



def get_clear_words(text): # Очистка слов
    spaced_text = "".join([c if c.isalpha() else ' ' for c in text])
    words = spaced_text.split()
    clear_words = []
    for w in words:
      if len(w) > 3:
        w = w.encode('ASCII', errors='ignore').decode('ASCII')
        w = bytes(w, 'ASCII').decode('ASCII', 'ignore')
        clear_words.append(letters(w))
    return clear_words


def read_text_from_file(file): # функция для открытия файлов
    with io.open(file, mode='r', encoding="utf-8") as f:
        return f.read()


def get_files_from_path(path):# функция для получения файлов
    arr = os.listdir(path)
    return arr

files = get_files_from_path(path_of_files)
for f in files: # проходимся по всем файлам и для каждого создаем список лемм и токенов
    print(f)
    if f > '0':
        text = read_text_from_file(path_of_files + f)
        words = get_clear_words(text)
        tokenized_map = {}
        lemmatize(words, tokenized_map)

        all_tokens = tokenized_map.values()
        with io.open('tokens/tokens_' + str(f), 'w', encoding="utf-8") as ff:
            all_tokens =  [item for sublist in all_tokens for item in sublist]
            all_tokens.sort()
            for t in all_tokens:
               ff.write(f"{t}\n")

        with io.open('lemmas/lemmas_' + str(f), 'w', encoding="utf-8") as ff:
            items = tokenized_map.items()
            items = sorted(items)
            for lemma in items:
                ff.write(f"{lemma}\n")

51.txt
34.txt
26.txt
76.txt
61.txt
15.txt
57.txt
41.txt
81.txt
29.txt
47.txt
87.txt
35.txt
72.txt
52.txt
65.txt
58.txt
24.txt
73.txt
48.txt
12.txt
5.txt
90.txt
30.txt
89.txt
18.txt
93.txt
46.txt
95.txt
94.txt
84.txt
66.txt
97.txt
44.txt
50.txt
74.txt
27.txt
14.txt
78.txt
9.txt
64.txt
54.txt
25.txt
96.txt
1.txt
36.txt
88.txt
79.txt
33.txt
63.txt
39.txt
100.txt
10.txt
32.txt
70.txt
21.txt
13.txt
82.txt
42.txt
4.txt
49.txt
55.txt
6.txt
86.txt
17.txt
77.txt
80.txt
68.txt
45.txt
56.txt
69.txt
31.txt
43.txt
11.txt
22.txt
62.txt
38.txt
59.txt
92.txt
2.txt
40.txt
8.txt
75.txt
28.txt
16.txt
19.txt
83.txt
7.txt
60.txt
67.txt
23.txt
37.txt
3.txt
91.txt
53.txt
85.txt
98.txt
71.txt
20.txt
99.txt


# Задание 3

In [4]:
import os
import sys
import json as j

json = dict()

directory = os.fsencode(r'texts')
count = 0
for file in os.listdir(directory):
    file_num = file.decode("ASCII").split('.')[0]

    if file_num > '0':
        with open(rf'texts/{file_num}.txt', 'r', encoding='utf-8') as file:
            data = file.read()

    for word in data.split(' '):
        if word in json:
            if not json.get(word).__contains__(file_num):
                json.get(word).append(file_num)
        else:
            json[word] = [file_num]

with open(rf'index2.json', 'w', encoding='utf-8') as file:
    j.dump(json, file, ensure_ascii=False)

print(len(json.keys()))
print('Success')

77919
Success


In [5]:
import ast
import sys

a = input()

arr = a.split(' ')

if len(arr) != 5:
    sys.exit('Некорректное выражение')

with open(r'index2.json', 'r',
          encoding='utf-8') as file:
    index = ast.literal_eval(file.read())

all_values = set(range(1, 101))
all_values = set([str(i) for i in all_values])

new_arr = []

for i in range(len(arr)):
    if i % 2 == 0:
        print(arr[i])
        if arr[i].startswith('NOT'):
            if index.get(arr[i][1:]) is None:
                sys.exit('Слова из выражения не существуют')
            new_arr.append(all_values.difference(index.get(arr[i][1:])))
            arr[i] = arr[i][1:]
        else:
            if index.get(arr[i]) is None:
                sys.exit('Слова из выражения не существуют')
            new_arr.append(index.get(arr[i]))

if arr[1] in {'OR'}:
    if len(new_arr) < 3:
        sys.exit('Некорректное выражение')
    if arr[3] in {'OR'}:
        result = set(new_arr[0]) & set(new_arr[1]) | set(new_arr[2])
    elif arr[3] in {'AND'}:
        result = set(new_arr[0]) & set(new_arr[1]) | set(new_arr[2])
elif arr[1] in {'AND'}:
    if len(new_arr) < 3:
        sys.exit('Некорректное выражение')
    if arr[3] in {'OR'}:
        result = set(new_arr[0]) & set(new_arr[1]) | set(new_arr[2])
    elif arr[3] in {'AND'}:
        result = set(new_arr[0]) & set(new_arr[1]) | set(new_arr[2])

if len(result) == 0:
    print("не найдено")
else:
    result = set([int(i) for i in result])
    print(sorted(result))

KeyboardInterrupt: Interrupted by user

# Загрузка файлов

In [26]:
!zip -r /content/texts.zip /content/texts
!zip -r /content/tokens.zip /content/tokens
!zip -r /content/lemmas.zip /content/lemmas

  adding: content/texts/ (stored 0%)
  adding: content/texts/51.txt (deflated 80%)
  adding: content/texts/34.txt (deflated 64%)
  adding: content/texts/26.txt (deflated 64%)
  adding: content/texts/76.txt (deflated 68%)
  adding: content/texts/61.txt (deflated 79%)
  adding: content/texts/15.txt (deflated 54%)
  adding: content/texts/57.txt (deflated 78%)
  adding: content/texts/41.txt (deflated 81%)
  adding: content/texts/81.txt (deflated 69%)
  adding: content/texts/29.txt (deflated 62%)
  adding: content/texts/47.txt (deflated 78%)
  adding: content/texts/87.txt (deflated 71%)
  adding: content/texts/35.txt (deflated 77%)
  adding: content/texts/72.txt (deflated 60%)
  adding: content/texts/52.txt (deflated 77%)
  adding: content/texts/65.txt (deflated 89%)
  adding: content/texts/58.txt (deflated 79%)
  adding: content/texts/24.txt (deflated 77%)
  adding: content/texts/73.txt (deflated 74%)
  adding: content/texts/48.txt (deflated 85%)
  adding: content/texts/12.txt (deflated 53

In [28]:
from google.colab import files
files.download("/content/lemmas.zip")
files.download("/content/tokens.zip")
files.download("/content/texts.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Задание 4

In [6]:
pip install unicodecsv

  Preparing metadata (setup.py) ... done
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10746 sha256=26396b972cf2546e8a1c0a132f5b57e3367e0e62f76665943342b3eeba92ed49
  Stored in directory: /root/.cache/pip/wheels/9c/ea/66/8e45247b09052a933eb1a680b7c64802298faba58aac9b346b
Successfully built unicodecsv


In [7]:
pip install methods

ERROR: Could not find a version that satisfies the requirement methods (from versions: none)
ERROR: No matching distribution found for methods


In [42]:
import math
import pymorphy3
import io
import re
from bisect import bisect_left
import unicodecsv as csv
import pymorphy3
import os
import re
import io
from bisect import bisect_left
import unicodecsv as csv
morph = pymorphy3.MorphAnalyzer()
from collections import Counter

files = {}

def search(a, x, lo=0, hi=None):
    if hi is None: hi = len(a)
    pos = bisect_left(a, x, lo, hi)
    return pos if pos != hi and a[pos] == x else -1

def find(target, L):
    if L in target:
        return True
    else:
        return False


def count_words(word, file):
    l = word.lower()
    if files.get(file) is not None:
        words = files.get(file)["words"]
        return find(words, l)


def check_lemmas(lemma, file):
    l = lemma.lower()
    if files.get(file) is not None:
        lemmas = files.get(file)["lemmas"]

        return find(lemmas, l)


def check_word_in_files(word, files):
    documents = []
    for f in files:
        if count_words(word, f) is True:
            documents.append(f)
    return documents


def check_lemma_in_files(lemma, files):
    documents = []
    for f in files:
        if check_lemmas(lemma, f) is True:
            documents.append(f)
    return documents


def lemmatize(words, tokenized_map):
    for word in words:
        lowered = word.lower()
        if pymorphy3.MorphAnalyzer().parse(word)[0].tag.POS(lowered) not in {'INTJ', 'PRCL', 'CONJ', 'PREP'}:
            p = morph.parse(word)[0].normal_form
            arr = tokenized_map.get(p)
            if arr is None:
                new_arr = [lowered]
                tokenized_map[p] = new_arr
            else:
                arr.append(lowered)


def get_clear_words_from_file(file):
    print(file)
    with io.open(file, mode='r', encoding="utf-8") as f:
        return f.read().split('\n')



def get_files_from_path(path):
    arr = os.listdir(path)
    return arr

morph = pymorphy3.MorphAnalyzer()
from collections import Counter

all_files = {}

def get_lemma_file_by_token_file(file):
    file_index = int(re.search(r'\d+', file).group())
    found = None
    for f in lemma_files:
        if re.search(r'\d+', f) is not None:
            f_index = int(re.search(r'\d+', f).group())
            if f_index == file_index:
                found = f
                break
    return found


files = get_files_from_path("/content/tokens")
lemma_files = get_files_from_path("/content/lemmas")

for f in files:
        tokenized_map = {}

        words = get_clear_words_from_file(f)

        lemma_file = get_lemma_file_by_token_file(f)
        lemmas = get_clear_words_from_file(lemma_file)
        all_files[f] = {}


for f in files:
    print(f)
    tokenized_map = {}
    try:
      words = all_files[f]["words"]
      lemmas = all_files[f]["lemmas"]
    except:
      continue

    lemmatize(words, tokenized_map)

    total_token_count = len(words)
    c = Counter(words)
    uniq_token = c.keys()

    with io.open("tokensTf/tf" + f + ".csv", 'w', encoding="utf-8") as tff:
        t_str = "TOKEN;TF;IDF;TF*IDF"
        tff.write(f"{t_str}\n")
        for t in uniq_token:
            tf = round((c[t] / total_token_count), 6)
            document_count_for_t = check_word_in_files(t, files)
            idf = round(math.log2(len(files) / len(document_count_for_t)), 6)
            tf_idf = round(tf * idf, 6)
            t_str = f"{t};{tf};{idf};{tf_idf}"
            tff.write(f"{t_str}\n")

    lemmas_count = len(tokenized_map.keys())

    with io.open("lemmaTf/lemma" + f + ".csv", 'w', encoding="utf-8") as tff:
        t_str = "LEMMA;TF;IDF;TF*IDF"
        tff.write(f"{t_str}\n")
        for l in lemmas:
            tf = round(len(tokenized_map[l]) / lemmas_count,6)
            document_count_for_l = check_lemma_in_files(l, files)
            idf = round(math.log2(len(files) / len(document_count_for_l)),6)
            tf_idf = round(tf * idf, 6)
            t_str = f"{l};{tf};{idf};{tf_idf}"
            tff.write(f"{t_str}\n")

tokens_70.txt
lemmas_70.txt
tokens_82.txt
lemmas_82.txt
tokens_27.txt
lemmas_27.txt
tokens_79.txt
lemmas_79.txt
tokens_16.txt
lemmas_16.txt
tokens_8.txt
lemmas_8.txt
tokens_60.txt
lemmas_60.txt
tokens_50.txt
lemmas_50.txt
tokens_17.txt
lemmas_17.txt
tokens_65.txt
lemmas_65.txt
tokens_88.txt
lemmas_88.txt
tokens_63.txt
lemmas_63.txt
tokens_89.txt
lemmas_89.txt
tokens_32.txt
lemmas_32.txt
tokens_80.txt
lemmas_80.txt
tokens_40.txt
lemmas_40.txt
tokens_23.txt
lemmas_23.txt
tokens_81.txt
lemmas_81.txt
tokens_56.txt
lemmas_56.txt
tokens_36.txt
lemmas_36.txt
tokens_34.txt
lemmas_34.txt
tokens_77.txt
lemmas_77.txt
tokens_64.txt
lemmas_64.txt
tokens_86.txt
lemmas_86.txt
tokens_67.txt
lemmas_67.txt
tokens_44.txt
lemmas_44.txt
tokens_28.txt
lemmas_28.txt
tokens_7.txt
lemmas_7.txt
tokens_62.txt
lemmas_62.txt
tokens_94.txt
lemmas_94.txt
tokens_46.txt
lemmas_46.txt
tokens_33.txt
lemmas_33.txt
tokens_20.txt
lemmas_20.txt
tokens_54.txt
lemmas_54.txt
tokens_49.txt
lemmas_49.txt
tokens_47.txt
lemmas_47.

In [31]:
rm -rf `find -type d -name .ipynb_checkpoints`